In [25]:
# Load the required dependencies
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor

from scipy import stats
from scipy.stats import linregress
from scipy.stats import f_oneway

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError


from pandas.plotting import scatter_matrix

pd.set_option('display.max_columns', 150, 'display.max_rows', 255)
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Read in the File and preview it.
file_path = Path("secondary_housing_data.csv")
df = pd.read_csv(file_path)
df.head()

,MSSubClass,LotArea,OverallCond,Exterior1st,TotalBsmtSF,CentralAir,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,GarageArea,Neighborhood,Age,SalePrice
0,60,8450,5,VinylSd,856,Y,1710,2,1,3,1,8,548,CollgCr,5,208500
1,20,9600,8,MetalSd,1262,Y,1262,2,0,3,1,6,460,Veenker,31,181500
2,60,11250,5,VinylSd,920,Y,1786,2,1,3,1,6,608,CollgCr,7,223500
3,70,9550,5,Wd Sdng,756,Y,1717,1,0,3,1,7,642,Crawfor,91,140000
4,60,14260,5,VinylSd,1145,Y,2198,2,1,4,1,9,836,NoRidge,8,250000


In [3]:
# Create a copy of the dataframe to use in neural networks
nn_df = df.copy()
# nn_df = nn_df.drop(columns=['Neighborhood'])
nn_df.head()

,MSSubClass,LotArea,OverallCond,Exterior1st,TotalBsmtSF,CentralAir,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,GarageArea,Neighborhood,Age,SalePrice
0,60,8450,5,VinylSd,856,Y,1710,2,1,3,1,8,548,CollgCr,5,208500
1,20,9600,8,MetalSd,1262,Y,1262,2,0,3,1,6,460,Veenker,31,181500
2,60,11250,5,VinylSd,920,Y,1786,2,1,3,1,6,608,CollgCr,7,223500
3,70,9550,5,Wd Sdng,756,Y,1717,1,0,3,1,7,642,Crawfor,91,140000
4,60,14260,5,VinylSd,1145,Y,2198,2,1,4,1,9,836,NoRidge,8,250000


In [4]:
# nn_df = nn_df.drop(columns='Id', axis=1)
nn_df.columns

Index(['MSSubClass', 'LotArea', 'OverallCond', 'Exterior1st', 'TotalBsmtSF',
       'CentralAir', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'GarageArea', 'Neighborhood', 'Age',
       'SalePrice'],
      dtype='object')

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy =  nn_df.dtypes[nn_df.dtypes == "object"].index.tolist()
df_dummies = pd.get_dummies(nn_df, columns=dummy)

df_dummies.head()

,MSSubClass,LotArea,OverallCond,TotalBsmtSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,GarageArea,Age,SalePrice,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,CentralAir_N,CentralAir_Y,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,60,8450,5,856,1710,2,1,3,1,8,548,5,208500,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20,9600,8,1262,1262,2,0,3,1,6,460,31,181500,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,60,11250,5,920,1786,2,1,3,1,6,608,7,223500,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,70,9550,5,756,1717,1,0,3,1,7,642,91,140000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,60,14260,5,1145,2198,2,1,4,1,9,836,8,250000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [6]:
# Calculate Z-values
df_dummies['price_scaled'] = np.abs(stats.zscore(df_dummies['SalePrice']))

# Filter out outliers
df_dummies = df_dummies[df_dummies['price_scaled'] <= 2.5]

# Drop the column
df_dummies = df_dummies.drop('price_scaled', axis=1)

In [7]:
df_dummies.describe()

,MSSubClass,LotArea,OverallCond,TotalBsmtSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,GarageArea,Age,SalePrice,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,CentralAir_N,CentralAir_Y,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
count,"1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00","1,420.00"
mean,57.28,"10,325.43",5.59,"1,033.47","1,484.81",1.55,0.37,2.86,1.05,6.44,463.19,37.33,"173,066.06",0.01,0.00,0.00,0.03,0.00,0.04,0.15,0.00,0.15,0.08,0.00,0.02,0.35,0.14,0.02,0.07,0.93,0.01,0.00,0.01,0.04,0.02,0.10,0.03,0.07,0.06,0.03,0.01,0.03,0.16,0.01,0.05,0.02,0.04,0.08,0.02,0.05,0.04,0.06,0.01,0.03,0.01
std,42.65,"9,899.16",1.11,413.12,486.66,0.54,0.50,0.81,0.22,1.56,206.86,30.10,"63,183.89",0.12,0.03,0.04,0.18,0.03,0.19,0.36,0.03,0.36,0.27,0.04,0.13,0.48,0.35,0.13,0.25,0.25,0.11,0.04,0.11,0.20,0.14,0.31,0.18,0.26,0.23,0.16,0.11,0.18,0.37,0.08,0.22,0.15,0.20,0.27,0.13,0.22,0.20,0.24,0.11,0.16,0.08
min,20.00,"1,300.00",1.00,0.00,334.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,"34,900.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,20.00,"7,500.00",5.00,792.75,"1,122.75",1.00,0.00,2.00,1.00,5.00,318.75,8.00,"129,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,50.00,"9,362.00",5.00,978.50,"1,443.50",2.00,0.00,3.00,1.00,6.00,473.00,36.00,"160,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,70.00,"11,376.25",6.00,"1,261.25","1,740.50",2.00,1.00,3.00,1.00,7.00,576.00,55.00,"207,125.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
max,190.00,"215,245.00",9.00,"6,110.00","5,642.00",3.00,2.00,8.00,3.00,14.00,"1,418.00",136.00,"378,500.00",1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [8]:
# Split our preprocessed data into our features and target arrays
y = df_dummies['SalePrice'].values
X = df_dummies.drop(['SalePrice'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
y_train.min()

34900

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
X_test_scaled[:5]

array([[-0.15071694, -0.39846324,  0.37015124, -0.5320186 , -0.63461758,
        -1.01896991, -0.74640987,  0.18408663, -0.211409  , -0.27565917,
         0.29911574,  1.15236324, -0.11952286, -0.02968261, -0.04199605,
        -0.18855106, -0.02968261, -0.19104018, -0.43963315, -0.02968261,
        -0.42529188, -0.28438593, -0.02968261, -0.13042181, -0.72869562,
         2.46981781, -0.13042181, -0.26396941,  0.26396941, -0.09423903,
        -0.04199605, -0.10332549,  4.86781801, -0.13723732, -0.3404608 ,
        -0.19350174, -0.26776503, -0.25232997, -0.16185465, -0.09423903,
        -0.19593679, -0.45381141, -0.0664896 , -0.22771002, -0.15603287,
        -0.21004201, -0.29332928, -0.11952286, -0.23195547, -0.20774711,
        -0.25035055, -0.11952286, -0.16185465, -0.08421519],
       [ 0.08741164,  0.11636039,  0.37015124, -0.80572745,  0.55773183,
        -1.01896991,  3.2913121 ,  0.18408663, -0.211409  ,  0.37946388,
         0.35331667,  0.05647808, -0.11952286, -0.02968261, -0.

In [12]:
from tensorflow.keras import backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  50
hidden_nodes_layer2 = 55
hidden_nodes_layer3 = 52

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2750      
                                                                 
 dense_1 (Dense)             (None, 55)                2805      
                                                                 
 dense_2 (Dense)             (None, 52)                2912      
                                                                 
 dense_3 (Dense)             (None, 1)                 53        
                                                                 
Total params: 8520 (33.28 KB)
Trainable params: 8520 (33.28 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss='mse', optimizer="adam", metrics=['accuracy', rmse])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
36/36 [==============================] - 1s 2ms/step - loss: 33999902720.0000 - accuracy: 0.0000e+00 - rmse: 184585.5625
Epoch 2/100
36/36 [==============================] - 0s 2ms/step - loss: 33989066752.0000 - accuracy: 0.0000e+00 - rmse: 184129.7188
Epoch 3/100
36/36 [==============================] - 0s 4ms/step - loss: 33925337088.0000 - accuracy: 0.0000e+00 - rmse: 183382.8750
Epoch 4/100
36/36 [==============================] - 0s 3ms/step - loss: 33685991424.0000 - accuracy: 0.0000e+00 - rmse: 182954.2812
Epoch 5/100
36/36 [==============================] - 0s 4ms/step - loss: 33019392000.0000 - accuracy: 0.0000e+00 - rmse: 181170.6562
Epoch 6/100
36/36 [==============================] - 0s 2ms/step - loss: 31543089152.0000 - accuracy: 0.0000e+00 - rmse: 177078.8594
Epoch 7/100
36/36 [==============================] - 0s 2ms/step - loss: 28823576576.0000 - accuracy: 0.0000e+00 - rmse: 169380.7188
Epoch 8/100
36/36 [==============================] - 0s 3ms/step - lo

In [16]:
# Evaluate the model using the test data
model_loss = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Evaluation: {model_loss}")

9/9 - 0s - loss: 945949888.0000 - accuracy: 0.0000e+00 - rmse: 29156.2500 - 234ms/epoch - 26ms/step
Evaluation: [945949888.0, 0.0, 29156.25]


In [17]:
y_pred = nn.predict(X_test)
# print(MeanSquaredError(y_test, y_pred).numpy())
# print(f'y_actual: {y_test}, y_pred:{y_pred}')
y_pred[:10]

9/9 [==============================] - 0s 1ms/step


array([[8.2781712e+07],
       [1.3653701e+08],
       [1.7279056e+08],
       [1.1250097e+08],
       [1.5405232e+08],
       [5.9047716e+07],
       [8.6634072e+07],
       [1.4596797e+08],
       [1.0861210e+08],
       [1.4573250e+08]], dtype=float32)

In [28]:
# # Step 1: Connect to MongoDB
# import pymongo
# client = pymongo.MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB connection string
# db = client["home_price_prediction"]  # Replace with your database name

In [29]:
# # Step 2: Insert data into MongoDB collection
# collection = db["df"]  # Replace with your collection name
# data_records = df.to_dict(orient='records')
# collection.insert_many(data_records)